## Notebook for augmenting images used for first stage YOLO model of 2-stage model pipline

In [11]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import itertools
from tqdm.notebook import tqdm
import augment_utils as aug
from concurrent.futures import ThreadPoolExecutor

aug.INPUT_DIR = 'TwoStageYOLODataset/original'
aug.OUTPUT_DIR = 'TwoStageYOLODataset/augmented'

AUG_FUNCTIONS = [
    aug.add_gaussian_noise,
    aug.adjust_random_brightness,
    aug.add_black_rect,
    aug.horizontal_flip,
    aug.random_rotate,
    aug.random_scale_with_padding,
]


# 获取所有图片名（不带扩展名）
input_images = [
    os.path.splitext(f)[0]
    for f in os.listdir(os.path.join(aug.INPUT_DIR, "images"))
    if f.endswith(".jpg")
]

# # 对每张图进行增强
# for filename in tqdm(input_images):
#     # 1. 原图保存
#     image, boxes, class_labels, _, _ = aug.load_image_and_boxes(filename)
#     aug.save_augmented(image, boxes, class_labels, filename)

#     # 2. 所有6选4组合增强
#     for combo in itertools.combinations(AUG_FUNCTIONS, 4):
#         # 从原图开始依次增强
#         temp_filename = filename
#         image, boxes, class_labels, _, _ = aug.load_image_and_boxes(temp_filename)

#         for func in combo:
#             # 修改函数以支持传 image/boxes/class_labels 是更复杂的工作，这里重新从文件加载（确保函数独立）
#             func(temp_filename)  # 每个函数会自动保存带 hash 的增强版本


# 每张图像的增强逻辑，供线程调用
def process_one_image(filename):
    try:
        # 保存原图
        image, boxes, class_labels, _, _ = aug.load_image_and_boxes(filename)
        aug.save_augmented(image, boxes, class_labels, filename)

        # 生成所有 6 选 4 的组合
        combos = list(itertools.combinations(AUG_FUNCTIONS, 4))
        for combo in combos:
            for func in combo:
                func(filename)  # 每个函数内部自己保存带 hash+timestamp 的结果
    except Exception as e:
        print(f"[ERROR] {filename} failed: {e}")

# 使用 ThreadPoolExecutor 加速
with ThreadPoolExecutor(max_workers=20) as executor:
    list(tqdm(executor.map(process_one_image, input_images), total=len(input_images)))

  0%|          | 0/138 [00:00<?, ?it/s]

c:\Users\Yuzhi\OneDrive\493Proj\CSE493G-Project\augment_utils.py:138: UserWarning: Argument(s) 'value' are not valid for transform Rotate
  A.Rotate(limit=30, border_mode=cv2.BORDER_CONSTANT, value=0, p=1.0),
c:\Users\Yuzhi\OneDrive\493Proj\CSE493G-Project\augment_utils.py:160: UserWarning: Argument(s) 'mode, cval' are not valid for transform Affine
  A.Affine(scale=scale_factor, fit_output=True, mode=cv2.BORDER_CONSTANT, cval=0),
c:\Users\Yuzhi\OneDrive\493Proj\CSE493G-Project\augment_utils.py:161: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=h, min_width=w, border_mode=cv2.BORDER_CONSTANT, value=0, p=1.0),
